<a href="https://colab.research.google.com/github/ParthibMudi/Malaria/blob/main/Malaria_CNN_1st_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
predictions = lenet_model.predict(test_dataset[0])

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense , Flatten , Conv2D , MaxPool2D , Dropout , InputLayer, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True,
                                  as_supervised=True,
                                  shuffle_files = True,
                                  split=['train'])

In [ ]:
# for data in dataset[0].take(1):
#   print(data)

#data preparation

In [ ]:
# TRAIN_RATIO = 0.8
# TEST_RATIO = 0.1
# VAL_RATIO= 0.1
# #learning perpes
# dataset = tf.data.Dataset.range(10)
# print(list(dataset.as_numpy_iterator()))
# print(len(dataset))
# train_dataset = dataset.take(int(TRAIN_RATIO * len(dataset)))
# print(list(train_dataset.as_numpy_iterator()))
# print(len(train_dataset))

In [ ]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))
  return train_dataset, val_dataset, test_dataset

In [ ]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO )
# print(list(train_dataset.take(1).as_numpy_iterator()),
#       list(val_dataset.take(1).as_numpy_iterator()), list(test_dataset.take(1).as_numpy_iterator()))


In [ ]:
IM_SIZE=224
def resize_rescale(image , label):
  return tf.image.resize(image , (IM_SIZE,IM_SIZE))/255.0 , label

In [ ]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)
train_dataset

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

#data visualization

In [ ]:
# for i , (image , label) in enumerate(train_dataset.take(16)):
#   ax = plt.subplot(4,4,i+1)
#   plt.imshow(image)
#   plt.title(dataset_info.features['label'].int2str(label))
#   plt.axis('off')

In [ ]:
BAECH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BAECH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:

val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BAECH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:

test_dataset = test_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BAECH_SIZE).prefetch(tf.data.AUTOTUNE)


#data proseccing

#Model

In [ ]:
lenet_model = tf.keras.models.Sequential([
    InputLayer(input_shape=(IM_SIZE,IM_SIZE,3)),

    Conv2D(filters=6,kernel_size=3, strides=1,padding='valid', activation='relu'),
    BatchNormalization(),

    MaxPool2D(pool_size=2,strides=2),
    Conv2D(filters=16,kernel_size=3, strides=1,padding='valid' ,activation='relu'),
    BatchNormalization(),

    MaxPool2D(pool_size=2,strides=2),
    Flatten(),
    Dense(20,activation='relu'),
    BatchNormalization(),

    Dense(10,activation='relu'),
    BatchNormalization(),

    Dense(1,activation='sigmoid')

])
lenet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 6)       168       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 6)       24        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 6)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 16)      880       
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 16)      64        
 chNormalization)                                                
                                                        

In [ ]:
lenet_model.compile(optimizer=Adam(learning_rate=0.01),loss=BinaryCrossentropy(),metrics='accuracy')

In [ ]:
history = lenet_model.fit(train_dataset , epochs=2, verbose=1)

Epoch 1/2
689/689 [==============================] - 774s 1s/step - loss: 0.3065 - accuracy: 0.8759
Epoch 2/2
689/689 [==============================] - 759s 1s/step - loss: 0.1926 - accuracy: 0.9375


In [ ]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
predictions = lenet_model.predict(test_dataset[0])

In [ ]:
print(history.history)

{'loss': [0.30649250745773315, 0.1926216185092926], 'accuracy': [0.8758958578109741, 0.9375396966934204]}


#Model Evaluation and Testing

In [ ]:
test_dataset = test_dataset.batch(1)

In [ ]:
test_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1917, in test_step
        y_pred = self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Inpu